In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers


2023-03-29 10:47:54.828910: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
seq_model = Sequential([
    layers.Flatten(input_shape=(28,28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
    
])

In a **functional API** layout, we follow the next 3 steps in order to define our model:
 - 1) Input: define input to the model
 - 2) Layers: define a set of interconnected layers on the input
 - 3) Model: define the model using the input and output layers

In [3]:
from tensorflow.keras.layers import Input

In [4]:
input = Input(shape=(28,28))

In [9]:
from tensorflow.keras.layers import Flatten, Dense, Concatenate

In [6]:
x = Flatten()(input)
x = Dense(128, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

In [7]:
from tensorflow.keras.models import Model

In [8]:
func_model = Model(inputs = input, outputs = predictions)

## Branching models

In [13]:
layer1 = Dense(32)
layer2_1 = Dense(32)(layer1)
layer2_2 = Dense(32)(layer1)
layer2_3 = Dense(32)(layer1)
layer2_4 = Dense(32)(layer1)
merge = Concatenate([layer2_1,layer2_2,layer2_3,layer2_4])

TypeError: Inputs to a layer should be tensors. Got '<keras.layers.core.dense.Dense object at 0x16ffd8cd0>' (of type <class 'keras.layers.core.dense.Dense'>) as input for layer 'dense_11'.